# Partie I: design d’ETL
L’objectif de cette partie est de produire des spécifications d’ETL. A partir de données sources et à l’aide d’outils de la communautés OHDSI, vous produirez un document technique permettant d’expliciter les transformations nécessaires pour adapter les données sources au format OMOP CDM v5.4. Ce document doit permettre à tout intégrateur d’implémenter un ETL de ces données sources sans ambigüité ni interprétations personnelles.

## Études de données synthétiques
Données téléchargées depuis https://synthea.mitre.org/downloads

### Analyse du rapport généré par WhiteRabbit

In [23]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl') # Ignorer le warning de manque de formatage du fichier excel

path = "ScanReport.xlsx" # Path vers le rapport
scan_report=pd.ExcelFile(path)
print("Le rapport contient " + str(len(scan_report.sheet_names)) + " feuilles excel :")
print(scan_report.sheet_names)

Le rapport contient 19 feuilles excel :
['Field Overview', 'Table Overview', 'conditions.csv', 'providers.csv', 'patients.csv', 'allergies.csv', 'observations.csv', 'devices.csv', 'procedures.csv', 'supplies.csv', 'payers.csv', 'careplans.csv', 'payer_transitions.csv', 'immunizations.csv', 'imaging_studies.csv', 'medications.csv', 'encounters.csv', 'organizations.csv', '_']


#### Feuille Field Overview

In [37]:
scan_report.parse('Field Overview').head(10)

,Table,Field,Description,Type,Max length,N rows,N rows checked,Fraction empty,N unique values,Fraction unique,Average,Standard Deviation,Min,25%,Median,75%,Max
0,conditions.csv,START,NaN,DATE,10.0,-1.0,8376.0,0.00000,5356.0,0.639446,2003-06-28,6.646630e+03,1916-12-28,1995-02-19,2011-08-26,2016-01-12,2020-04-25
1,conditions.csv,STOP,NaN,DATE,10.0,-1.0,8376.0,0.45499,2924.0,0.349093,2012-03-13,3.839641e+03,1917-09-06,2011-10-29,2014-09-17,2017-06-15,2020-04-27
2,conditions.csv,PATIENT,NaN,VARCHAR,36.0,-1.0,8376.0,0.00000,1152.0,0.137536,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,conditions.csv,ENCOUNTER,NaN,VARCHAR,36.0,-1.0,8376.0,0.00000,7650.0,0.913324,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,conditions.csv,CODE,NaN,INT,15.0,-1.0,8376.0,0.00000,129.0,0.015401,1949885905490.615234,2.219533e+13,1734006.0,53741008.0,127013003.0,283371005.0,368581000119106.0
5,conditions.csv,DESCRIPTION,NaN,VARCHAR,80.0,-1.0,8376.0,0.00000,129.0,0.015401,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,providers.csv,Id,NaN,VARCHAR,36.0,-1.0,5855.0,0.00000,5855.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,providers.csv,ORGANIZATION,NaN,VARCHAR,36.0,-1.0,5855.0,0.00000,1119.0,0.191119,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,providers.csv,NAME,NaN,VARCHAR,29.0,-1.0,5855.0,0.00000,5766.0,0.984799,NaN,NaN,NaN,NaN,NaN,NaN,NaN


La table résume les champs présentés dans les autres feuilles, on retrouve bien les 16 noms de fichiers csv, plus la ligne "NaN" qui sépare les différentes tables.